In [1]:
import pymorphy2
import nltk
import ujson as json
import matplotlib.pyplot as plt
import numpy as np
import itertools
import gzip
import operator

from gensim.corpora.dictionary import Dictionary
from gensim.models import word2vec
from scipy.spatial.distance import cosine
from datetime import datetime
from collections import Counter

morth_analyzer = pymorphy2.MorphAnalyzer()

In [2]:
def split_words_v3(a_text):
    cur_word = ''
    prev_is_alpha = False

    for letter in a_text:
        if  (letter.isalpha() and prev_is_alpha or 
            letter.isdigit() and not prev_is_alpha):
            cur_word += letter
        elif (letter.isalpha() and not prev_is_alpha or
             letter.isdigit() and prev_is_alpha):
            if cur_word: yield cur_word
            cur_word = letter
            prev_is_alpha = not prev_is_alpha
        else:
            if cur_word: yield cur_word
            cur_word = ''
            prev_is_alpha = False
    if cur_word: yield cur_word
        
MORTH_CACHE = {}
def get_norm_word_v3(a_word):
    if a_word not in MORTH_CACHE: MORTH_CACHE[a_word] = morth_analyzer.parse(a_word)[0].normal_form
    return MORTH_CACHE[a_word]

def get_doc_words(a_doc, a_split=split_words_v3, a_norm_word=get_norm_word_v3):
    for word in itertools.chain(a_split(a_doc.title), a_split(a_doc.description)):
        yield a_norm_word(word)
        
def get_doc_words_(a_doc, a_split=split_words_v3, a_norm_word=get_norm_word_v3):
    for word in itertools.chain(a_split(a_doc.title), a_split(a_doc.description)):
        yield a_norm_word(word)

In [3]:
class Document:
    def __init__(self, init_dict):
        self.title = init_dict.get('title', '')
        self.description = init_dict.get('description', '')
        self.url = init_dict.get('url', '')
        self.site = init_dict.get('site', '')
        self.ts = datetime.fromtimestamp(init_dict['ts']) if 'ts' in init_dict else -1
        self.words = []
        for word in split_words_v3(self.title):
            self.words +=  word
        for word in split_words_v3(self.description):
            self.words +=  word
    
    def __str__(self):
        res = ''
        res += 'url : %s\n' % self.url
        res += 'date : %s\n' % self.ts
        res += 'title : %s\n' % self.title
        res += 'description : %s\n' % self.description
        res += 'site : %s\n' % self.site
        return res

In [4]:
fin = gzip.open('dataset_mai.jsonl.gz')
for line in itertools.islice(fin, 10):
    data = json.loads(line.strip())
    print(Document(data))

url : http://bloknot-volzhsky.ru/news/volzhane-mogut-podat-zayavlenie-na-letnie-putevki-
date : 2019-11-30 18:26:10
title : Волжане могут подать заявление на летние путевки для детей
description : С понедельника заявления начинают принимать в МФЦ
site : bloknot-volzhsky.ru

url : https://trikky.ru/test-na-znanie-russkogo-yazyka-423354.html
date : 2019-11-30 18:26:48
title : 💗Тест на знание русского языка💗
description : Тест со сложными и легкими вопросами. Для кого-то будет легко набрать все 100 баллов, а кому-то будет немного тяжело. В любом случае попробовать стоит.1. Что изучает фразеология? способы образования слов устойчивые сочетания слов части речи2. На месте каких цифр в словах пишется н? В простенке между занавеше(1)ыми окнами были установле(2)ы часы, а рядом с ними […]
site : trikky.ru

url : https://topwar.ru/165315-chernomorskij-flot-popolnilsja-150-tonnym-plavkranom-proekta-02690.html
date : 2019-11-30 18:26:43
title : Черноморский флот пополнился 150-тонным плавкраном про

In [5]:
fin = gzip.open('dataset_mai.jsonl.gz')
dataset = []
dataset_test = []
for line in itertools.islice(fin, 10000):
    data = json.loads(line.strip())
    dataset.append(Document(data))

## ДЗ - реализовать поиск похожих документов по текстовым векторам и по word2vec векторам

In [125]:
def make_set(a_doc, a_data):
    doc_words = list(get_doc_words(a_doc))
    data_words = list(get_doc_words(a_data))
    c = list(set(doc_words) & set(data_words))
    return c

def get_word_match_most_similar_docs(a_doc, a_dataset, a_top_n=10):
    print("\nGET_WORD_MATCH_MOST_SIMILAR_DOCS FUNCTION \n")
    print("original doc: {}".format(a_doc))
    print("**************************************************************************************************************")
    l = len(set(get_doc_words(a_doc)))
    for item in itertools.islice(sorted(a_dataset, key=lambda x: len(make_set(a_doc, x)), reverse=True), a_top_n):
        print("Similar: {}".format(len(make_set(a_doc, item)) / l))
        print(item)
        
#################################################################################################################
def get_dict(a_dataset):
    for item in a_dataset:
        item.words = list(get_doc_words(item))
    res = Dictionary(item.words for item in a_dataset)
    return res

def get_doc_vec(a_doc, a_dictionary):
    indexs_word = a_dictionary.doc2idx(a_doc.words)
    res = np.zeros(len(a_dictionary))
    for i in indexs_word:
        res[i] += 1
    return res

def get_tf_idf_most_similar_doc(a_doc, a_dataset, a_top_n=10):
    # для каждого документа строится вектор размерности словаря (аналогично random forest) и добавляется idf
    # далее cosine между документами
    print("\nGET_TF_IDF_MOST_SIMILAR_DOC FUNCTION \n")
    print(f'original: \n{a_doc}')
    print("**************************************************************************************************************")
    dictionary = get_dict(a_dataset)
    doc_vec = get_doc_vec(a_doc, dictionary)
    for item in itertools.islice(sorted(a_dataset, key=lambda x: cosine(doc_vec, get_doc_vec(x, dictionary))), a_top_n):
        print(item)

#################################################################################################################
def get_vec_w2v(a_doc, a_data_w2v):
    res = np.zeros(100)
    for word in a_doc.words:
        if word in a_data_w2v:
            res = res + a_data_w2v[word].copy()
    return res / len(a_doc.words)

def get_w2v_most_similar_doc(a_doc, a_dataset, a_top_n=10):
    # считается средний вектор по всем словам (можно при усреднении учитывать idf) cosine(x, doc_w2v)
    print("\nGET_W2V_MOST_SIMILAR_DOC FUNCTION \n")
    print("original doc: {}".format(a_doc))
    print("**************************************************************************************************************")
    for item in a_dataset:
        item.words = list(get_doc_words(item))
    data_w2v = word2vec.Word2Vec([item.words for item in a_dataset], workers=4)
    doc_w2v = get_vec_w2v(a_doc, data_w2v)
    for item in itertools.islice(sorted(a_dataset, key=lambda x: cosine(doc_w2v, get_vec_w2v(x, data_w2v))), a_top_n):
        print(item)

In [9]:
fin = gzip.open('dataset_mai.jsonl.gz')
dataset_w2v = []
dataset = []
for line in itertools.islice(fin, 100000):
    data = json.loads(line.strip())
    dataset.append(Document(data))
    dataset_w2v.append(Document(data))

#### Тестовые данные

In [ ]:
doc_id = 13 #dota2
doc_id = 1946 #гороскоп
doc_id = 3388 #хоккей
doc_id = 7601 #телефоны

In [126]:
doc_id = 13 
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)


GET_WORD_MATCH_MOST_SIMILAR_DOCS FUNCTION 

original doc: url : https://cyber.sports.ru/dota2/1080755627.html
date : 2019-11-30 18:26:39
title : Результаты Parimatch League Dota 2. Virtus.pro победила
description : 30 ноября завершился турнир Parimatch League. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Лан-финал Parimatch League прошел с 28 по 30 ноября в Москве. 4 команды разыграли 70 тысяч долларов. Результаты команд 1. Virtus.pro2.
site : cyber.sports.ru

**************************************************************************************************************
Similar: 1.0
url : https://cyber.sports.ru/dota2/1080755627.html
date : 2019-11-30 18:26:39
title : Результаты Parimatch League Dota 2. Virtus.pro победила
description : 30 ноября завершился турнир Parimatch League. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Лан-финал Parimatch League прошел с 28 по 30 ноября в Москве. 4 команды ра

/home/ann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/home/ann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


url : https://cyber.sports.ru/dota2/1080755627.html
date : 2019-11-30 18:26:39
title : Результаты Parimatch League Dota 2. Virtus.pro победила
description : 30 ноября завершился турнир Parimatch League. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Лан-финал Parimatch League прошел с 28 по 30 ноября в Москве. 4 команды разыграли 70 тысяч долларов. Результаты команд 1. Virtus.pro2.
site : cyber.sports.ru

url : https://cyber.sports.ru/dota2/1080757051.html
date : 2019-11-30 18:37:09
title : Призовой фонд Parimatch League Dota 2
description : 30 ноября в Москве завершился турнир Parimatch League. Призовой фонд турнира – 70 тысяч долларов. В финале Virtus.pro разгромила HellRaisers со счетом 3:0 и заработала 40 тысяч долларов. Распределение призовых 1. Virtus.pro – $40 0002. HellRaisers – $20 0003.
site : cyber.sports.ru

url : https://cyber.sports.ru/dota2/1080752587.html
date : 2019-11-30 16:57:20
title : Virtus.pro ведет со счетом 2:0 по карта

In [127]:
doc_id = 1946 
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)


GET_WORD_MATCH_MOST_SIMILAR_DOCS FUNCTION 

original doc: url : https://www.obozrevatel.com/astro/news/goroskop-na-1-dekabrya-chto-zhdet-lvov-rakov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-30 17:07:11
title : Гороскоп на 1 декабря: что ждет Львов, Раков, Дев и другие знаки зодиака
description : Советы астрологов на воскресенье
site : obozrevatel.com

**************************************************************************************************************
Similar: 1.0
url : https://www.obozrevatel.com/astro/news/goroskop-na-1-dekabrya-chto-zhdet-lvov-rakov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-30 17:07:11
title : Гороскоп на 1 декабря: что ждет Львов, Раков, Дев и другие знаки зодиака
description : Советы астрологов на воскресенье
site : obozrevatel.com

Similar: 0.8125
url : https://www.obozrevatel.com/astro/news/goroskop-na-29-noyabrya-chto-zhdet-rakov-lvov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-29 00:36:09
title : Гороскоп на 29 ноября: что ждет Раков, Львов

url : https://www.obozrevatel.com/astro/news/goroskop-na-1-dekabrya-chto-zhdet-lvov-rakov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-30 17:07:11
title : Гороскоп на 1 декабря: что ждет Львов, Раков, Дев и другие знаки зодиака
description : Советы астрологов на воскресенье
site : obozrevatel.com

url : https://www.obozrevatel.com/astro/news/goroskop-na-29-noyabrya-chto-zhdet-rakov-lvov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-29 00:36:09
title : Гороскоп на 29 ноября: что ждет Раков, Львов, Дев и другие знаки зодиака
description : Советы астрологов на пятницу
site : obozrevatel.com

url : https://www.obozrevatel.com/astro/news/goroskop-na-28-noyabrya-chto-zhdet-rakov-lvov-dev-i-drugie-znaki-zodiaka1.htm
date : 2019-11-27 23:02:09
title : Гороскоп на 28 ноября: что ждет Раков, Львов, Дев и другие знаки зодиака
description : Советы астрологов на четверг
site : obozrevatel.com

url : https://www.obozrevatel.com/astro/news/goroskop-na-28-noyabrya-po-kartam-taro-chto-zhdet-ovnov-dev-

/home/ann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/home/ann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


url : https://www.obozrevatel.com/astro/news/goroskop-na-1-dekabrya-chto-zhdet-lvov-rakov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-30 17:07:11
title : Гороскоп на 1 декабря: что ждет Львов, Раков, Дев и другие знаки зодиака
description : Советы астрологов на воскресенье
site : obozrevatel.com

url : https://www.obozrevatel.com/astro/news/goroskop-na-28-noyabrya-chto-zhdet-rakov-lvov-dev-i-drugie-znaki-zodiaka1.htm
date : 2019-11-27 23:02:09
title : Гороскоп на 28 ноября: что ждет Раков, Львов, Дев и другие знаки зодиака
description : Советы астрологов на четверг
site : obozrevatel.com

url : https://www.obozrevatel.com/astro/news/goroskop-na-29-noyabrya-chto-zhdet-rakov-lvov-dev-i-drugie-znaki-zodiaka.htm
date : 2019-11-29 00:36:09
title : Гороскоп на 29 ноября: что ждет Раков, Львов, Дев и другие знаки зодиака
description : Советы астрологов на пятницу
site : obozrevatel.com

url : https://www.newsler.ru/society/2019/11/30/skorpiony-budut-otdelyat-zerna-ot-plevel-a-telcy-poznayut

In [128]:
doc_id = 3388 
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)


GET_WORD_MATCH_MOST_SIMILAR_DOCS FUNCTION 

original doc: url : https://www.tv21.ru/news/2019/11/30/khokkeisty-murmana-proveli-pervyy-domashniy-match-na-stadione-stroitel
date : 2019-11-30 15:31:47
title : Хоккеисты "Мурмана" провели первый домашний матч на стадионе "Строитель"
description : Игра состоялась в рамках открытия нового сезона Суперлиги по хоккею с мячом.
site : tv21.ru

**************************************************************************************************************
Similar: 1.0
url : https://www.tv21.ru/news/2019/11/30/khokkeisty-murmana-proveli-pervyy-domashniy-match-na-stadione-stroitel
date : 2019-11-30 15:31:47
title : Хоккеисты "Мурмана" провели первый домашний матч на стадионе "Строитель"
description : Игра состоялась в рамках открытия нового сезона Суперлиги по хоккею с мячом.
site : tv21.ru

Similar: 0.8095238095238095
url : https://komiinform.ru/news/189633
date : 2019-11-30 16:06:42
title : "Строитель" опробовал эжвинский лед
description : Сыктывка

url : https://www.tv21.ru/news/2019/11/30/khokkeisty-murmana-proveli-pervyy-domashniy-match-na-stadione-stroitel
date : 2019-11-30 15:31:47
title : Хоккеисты "Мурмана" провели первый домашний матч на стадионе "Строитель"
description : Игра состоялась в рамках открытия нового сезона Суперлиги по хоккею с мячом.
site : tv21.ru

url : https://vk.com/@mlive51-v-murmansk-vozvraschaetsya-bolshoi-hokkei
date : 2019-11-29 18:37:54
title : В Мурманск возвращается «большой хоккей»
description : 30 ноября хоккейный клуб «Мурман» встретится с ульяновской «Волгой» на стадионе «Строитель» в рамках Чемпионата России по хоккею с мячом Суперлиги сезона 2019/2020.
site : vk.com

url : https://b-port.com/news/234153
date : 2019-11-29 15:17:15
title : Стадион «Строитель» готов принять первую игру «Мурмана» в Суперлиге
description : На домашней арене мурманского хоккейного клуба были проведены работы по модернизации
site : b-port.com

url : https://severpost.ru/read/87523
date : 2019-11-29 14:21:10
title :

/home/ann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/home/ann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


url : https://www.tv21.ru/news/2019/11/30/khokkeisty-murmana-proveli-pervyy-domashniy-match-na-stadione-stroitel
date : 2019-11-30 15:31:47
title : Хоккеисты "Мурмана" провели первый домашний матч на стадионе "Строитель"
description : Игра состоялась в рамках открытия нового сезона Суперлиги по хоккею с мячом.
site : tv21.ru

url : https://mgimo.ru/about/news/social/mini-futbol-boevaya-nichya-s-akademiey-mchs/
date : 2019-11-29 14:19:06
title : Мини-футбол: боевая ничья с Академией МЧС
description : 28 ноября сборная МГИМО по мини-футболу в рамках Московских студенческих спортивных игр провела матч на выезде с командой АГПС. С первой до последней секунды шла бескомпромиссная борьба на каждом сантиметре поля. Благодаря надежной игре вратаря сборной МГИМО Э.Якубова матч закончился со счетом 5:5. Следующий матч состоится уже 4 декабря. Наша команда принимает команду МГПУ. Кафедра физического вос
site : mgimo.ru

url : https://vk.com/@hcmetallurg-metallurg-salavat-ulaev-ob-igre
date : 2019

In [129]:
doc_id = 7601
for similart_func in [get_word_match_most_similar_docs,  get_tf_idf_most_similar_doc, get_w2v_most_similar_doc]:
    similart_func(dataset[doc_id], dataset)


GET_WORD_MATCH_MOST_SIMILAR_DOCS FUNCTION 

original doc: url : https://megaobzor.com/Stala-izvestna-cena-smartfona-Redmi-K30.html
date : 2019-11-30 12:12:16
title : Стала известна цена смартфона Redmi K30
description : Авторитетный искатель утечек Мукул Шарма поделился подробностями о цене смартфона Redmi K30, официальный анонс которого состоится уже 10 декабря. Если верить источнику, аппарат обойдется в 327 долларов, что намного больше 285 долларов, которые ему приписывали ранее. По предварительным данным, Redmi K30 получит аккумулятор ёмкостью 5000 мАч, квадрокамеру с главным датчиком изображения разрешением 60 Мп, дисплей с частотой обновления 120 Гц, двойную фронтальную камеру на 20 Мп и 2 Мп и восьмиядерный процессор Snapdragon 730G.
site : megaobzor.com

**************************************************************************************************************
Similar: 1.0
url : https://megaobzor.com/Stala-izvestna-cena-smartfona-Redmi-K30.html
date : 2019-11-30 12:12:16
tit

url : https://megaobzor.com/Stala-izvestna-cena-smartfona-Redmi-K30.html
date : 2019-11-30 12:12:16
title : Стала известна цена смартфона Redmi K30
description : Авторитетный искатель утечек Мукул Шарма поделился подробностями о цене смартфона Redmi K30, официальный анонс которого состоится уже 10 декабря. Если верить источнику, аппарат обойдется в 327 долларов, что намного больше 285 долларов, которые ему приписывали ранее. По предварительным данным, Redmi K30 получит аккумулятор ёмкостью 5000 мАч, квадрокамеру с главным датчиком изображения разрешением 60 Мп, дисплей с частотой обновления 120 Гц, двойную фронтальную камеру на 20 Мп и 2 Мп и восьмиядерный процессор Snapdragon 730G.
site : megaobzor.com

url : https://megaobzor.com/Redmi-ne-stali-zamorachivatsja-s-reklamoi-Redmi-K30.html
date : 2019-11-28 14:06:22
title : Redmi не стали заморачиваться с рекламой Redmi K30
description : Бренд Redmi начал рекламную кампанию смартфона Redmi K30, который будет официально представлен 10 дек

/home/ann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/home/ann/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


url : https://megaobzor.com/Redmi-ne-stali-zamorachivatsja-s-reklamoi-Redmi-K30.html
date : 2019-11-28 14:06:22
title : Redmi не стали заморачиваться с рекламой Redmi K30
description : Бренд Redmi начал рекламную кампанию смартфона Redmi K30, который будет официально представлен 10 декабря. Забавно, что тизер практически идентичен рекламе предшественника Redmi K20 – боксерские перчатки и надпись о готовности отправить в нокаут конкурентов. По предварительным данным, Redmi K30 получит аккумулятор ёмкостью 5000 мАч, квадрокамеру с главным датчиком изображения разрешением 60 Мп, дисплей с частотой обновления 120 Гц, двойную фронтальную камеру на 20 Мп и 2 Мп и восьмиядерный процессор Snapdragon 730G. Цена версии с 6 ГБ оперативной и 64 ГБ флэш-памяти без поддержки сетей пятого поколения составит всего 285 долларов. Цена варианта с модемом 5G пока не называется.
site : megaobzor.com

url : https://megaobzor.com/Smartfon-Vivo-X30-pokazali-na-oficialnih-renderah.html
date : 2019-11-29 16:02: